<a href="https://colab.research.google.com/github/csch7/CSCI-4170/blob/main/Homework-03/Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Neural Networks
### Author: Colin Scherer

For this assignment, I've chosen a dataset of diabetes patients -- I'm trying to determine whether each patient will be readmitted to the hospital, and how soon. Although that sounds like a mix of classification and regression, this is in reality a multi-class classification problem with 3 classes: readmitted within 30 days, readmitted after 30 days, and not readmitted. As this dataset is very messy, I will have to do extensive preprocessing: any feature with more than 2% of values missing I will just remove -- this isn't too bad, since this only impacts features which intuitively wouldn't have much of an impact on the problem. I additionally convert all strings to an integer I thought made sense. Performing one-hot encoding on the categorical features would also be advised, but I ran out of time to do so. The targets of this dataset are also very imbalanced, with only 10% of the dataset being classified as one of 3 targets, and 60% of the dataset being classified as another. To solve this issue, I will downsample the larger portions randomly. I can afford to do this since the raw dataset has about 100k samples.

Even with these problems addressed, the dataset in general is messy, with plenty of missing data and unbalanced variables. For example, I am generalizing "NO" for "readmitted" to mean the patient was never readmitted. In reality, a "NO" means there was no record of readmission -- many of these could just be records that were lost. Due to the bad quality of data, I'm not expecting to get high accuracies -- any accuracies greater than ~40% will be considered a win.

In [316]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from google.colab import drive

def repl_age(age: str) -> int:
  return int(re.findall(r'\d+', age)[0]) # Converts age string into an integer by taking the lower bound of the range.

drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/diabetic_data.csv")

for c in df.columns:
  pctUnknown = np.sum([df[c] == '?'])/df[c].shape[0]
  if(pctUnknown > 0.02):
    df.drop(c, axis = 1, inplace = True)
  else:
    df = df[df[c] != '?']

# Transform strings into integers
df.drop('diag_1', axis = 1, inplace = True)
df.drop('diag_2', axis = 1, inplace = True)
df.drop('diag_3', axis = 1, inplace = True)
df.replace('Male', 0, inplace = True)
df.replace('Female', 1, inplace = True)
df = df[df['gender'] != 'Unknown/Invalid']
df.replace('No', 0, inplace = True)
df.replace('NO', 2, inplace = True)
df.replace('>30', 1, inplace = True)
df.replace('<30', 0, inplace = True)
df.replace('Down', 1, inplace = True)
df.replace('Steady', 2, inplace = True)
df.replace('Up', 3, inplace = True)
df.replace('Yes', 1, inplace = True)
df.replace('Ch', 1, inplace = True)
df.replace('Norm', 1, inplace = True)
df.replace('>7', 2, inplace = True)
df.replace('>8', 3, inplace = True)
df.replace('>200', 2, inplace = True)
df.replace('>300', 3, inplace = True)
df.fillna(0, inplace = True)
df['age'] = df['age'].apply(repl_age)

tars = df['readmitted']
df.drop('readmitted', axis = 1, inplace = True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-316-d281ce0916d6>:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('No', 0, inplace = True)
<ipython-input-316-d281ce0916d6>:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('<30', 0, inplace = True)
<ipython-input-316-d281ce0916d6>:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downc

In [318]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

dropped_features = []

for i, feature in enumerate(df.columns):
  vif = variance_inflation_factor(df, i)
  print("{f}:\t\t\t{v}".format(f=feature, v=vif))
  if vif > 5 or np.isnan(vif): # Remove any features with vif > 5.
    dropped_features.append(feature)

df.drop(dropped_features, axis = 1, inplace = True)
df.drop('patient_nbr', axis = 1, inplace = True)

patient_nbr:			2.6187727219405685
gender:			1.9767106176361124
admission_type_id:			3.0537541754831503
discharge_disposition_id:			1.5493688982755114
admission_source_id:			2.9887910431030305
time_in_hospital:			3.1912202094514175
num_procedures:			1.7395012916504649
number_outpatient:			1.1180938194445622
number_emergency:			1.1414915515245831
number_inpatient:			1.3781828685044712
max_glu_serum:			1.3918876293956812
A1Cresult:			1.1946062676438023
metformin:			1.4459896309297542
repaglinide:			1.0325326059033992
nateglinide:			1.0142990577027875
chlorpropamide:			1.0018860873801498
glimepiride:			1.110445304801677
acetohexamide:			1.0002580291888405
glipizide:			1.2615416515890447
glyburide:			1.242272289756267
tolbutamide:			1.0005351101108333
pioglitazone:			1.1547646225920742
rosiglitazone:			1.135545478683643
acarbose:			1.0073255839623099
miglitol:			1.0014137360291113
troglitazone:			1.0003599762364608
tolazamide:			1.0010978953284206
insulin:			2.687638722216703
glyburide-metf

In [322]:
def load_dataset(dat, tar):
  vals, cts = np.unique(tar, return_counts=True)
  min_val = vals[np.argmin(cts)]
  min_ct = min(cts)
  new_dat = dat[tar == min_val]
  new_tar = tar[tar == min_val]
  for i, v in enumerate(vals):
    if v != min_val:
      indices = list(np.where(tar == v))[0]
      new_ind = np.random.choice(indices, min_ct)
      new_dat = np.append(new_dat, dat[new_ind], axis=0)
      new_tar = np.append(new_tar, tar[new_ind])
  return new_dat, new_tar


def ReLu(z):
  return np.maximum(z, 0)



class myNeuralNetwork:
  def __init__(self, train_data, train_targets, epochs, num_hidden_layers=2, hidden_dim=16, lr=0.01, batch_size = 32):
    self.batch_size = batch_size
    self.train_data = train_data
    self.train_targets = train_targets
    self.epochs = epochs
    self.learning_rate = lr
    self.hidden_layers = num_hidden_layers
    self.weights = [np.random.uniform(-0.1, 0.1, (np.shape(train_data)[1], hidden_dim))]
    self.biases = [np.random.uniform(-0.1, 0.1, (hidden_dim, 1))]
    for i in range(1, num_hidden_layers):
      self.weights.append(np.random.uniform(-0.1, 0.1, (hidden_dim, hidden_dim)))
      self.biases.append(np.random.uniform(-0.1, 0.1, (hidden_dim, 1)))
    self.weights.append(np.random.uniform(-0.1, 0.1, (hidden_dim, np.shape(train_targets)[1])))
    self.biases.append(np.random.uniform(-0.1, 0.1, (np.shape(train_targets)[1], 1)))

  def forward(self, data):
    z = []
    a = [data.T]
    for l in range(self.hidden_layers+1):
      z.append(np.matmul(self.weights[l].T, a[l]) + self.biases[l])
      # print(a[-1])
      if(l != self.hidden_layers):
        a.append(ReLu(z[l]))
      else:
        a.append(np.exp(z[l])/np.sum(np.exp(z[l]), axis = 0))
    return a, z

  def backward(self, a, z, tars):
    deltas = [0] * (self.hidden_layers+1)
    # Calculate gradient for the last layer. The derivative of softmax is given by p_i - y_i
    deltas[-1] = (a[-1]-tars.T).T
    for l in range(self.hidden_layers-1, -1, -1):
      deltas[l] = np.matmul(deltas[l+1], self.weights[l+1].T)*(np.where(z[l] > 0, 1, 0)).T
    return deltas


  def train(self):
    N = np.shape(self.train_data)[0]
    for e in range(self.epochs):
      print("Epoch {}".format(e))
      for batch in range(np.shape(self.train_data)[0]//self.batch_size):
        a, z = self.forward(self.train_data[self.batch_size*batch:self.batch_size*(batch+1)])
        dels = self.backward(a, z, self.train_targets[self.batch_size*batch:self.batch_size*(batch+1)])

        for l in range(self.hidden_layers+1):
          self.weights[l] -= self.learning_rate*(np.matmul(dels[l].T, a[l].T)).T
          self.biases[l] = self.biases[l]-self.learning_rate*np.array([np.mean(dels[l].T, axis=1)]).T
        if(batch % 100 == 0):
          print("Loss: {}".format(self.cost(a[-1], self.train_targets[self.batch_size*batch:self.batch_size*(batch+1)])))


  def cost(self, preds, tars):
    return np.sum(-np.log(preds.T[range(np.shape(preds)[1]), np.argmax(tars, axis=1)]))/np.shape(preds)[1]

  def predict(self, data):
    probs, _ = self.forward(data)
    probs = probs[-1]
    return np.argmax(probs.T, axis=1)


For implementing a 2-layer neural network with pytorch, I closely followed [this tutorial](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html).

In [274]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

class torchNeuralNetwork(nn.Module):
  def __init__(self, num_features, num_classes, num_hidden_layers=2, hidden_dim=16):
    super().__init__()
    self.layer_list = [nn.BatchNorm1d(num_features), nn.Linear(num_features, hidden_dim), nn.Tanh()]
    for l in range(1, num_hidden_layers):
      self.layer_list.append(nn.Linear(hidden_dim, hidden_dim))
      self.layer_list.append(nn.Tanh())
    self.layer_list.append(nn.Linear(hidden_dim, num_classes))
    self.layer_list.append(nn.Softmax())
    self.layers = nn.Sequential(*self.layer_list)

  def forward(self, x):
    return self.layers(x)


For my training-validation-test split, I've chosen to take half of my dataset for training and sample validation and test sets at a 3:1:1 train:valid:test ratio from the rest of the dataset.

In [324]:
dat = np.array(df)
tar = np.array(tars)
dat, tar = load_dataset(dat, tar)
print(np.shape(dat))
N = np.shape(dat)[0]

pct_train = 0.5
pct_val = pct_train + pct_train*0.3
pct_test = pct_val + pct_train*0.3
indices = np.arange(N)
np.random.shuffle(indices)

targets = np.zeros((N, 3))
targets[range(N), tar] = 1

train_data = dat[indices[:int(pct_train*N)]]
train_targets = targets[indices[:int(pct_train*N)]]
val_data = dat[indices[int(pct_train*N):int(pct_val*N)]]
val_targets = targets[[indices[int(pct_train*N):int(pct_val*N)]]]
test_data = dat[indices[int(pct_val*N):int(pct_test*N)]]
test_targets = targets[indices[int(pct_val*N):int(pct_test*N)]]
mynn = myNeuralNetwork(train_data, train_targets, 100, lr=0.01)
mynn.train(val_data, val_targets)
preds = mynn.predict(test_data)
print(preds)
print(np.sum(test_targets[range(np.shape(test_targets)[0]), preds])/np.shape(test_targets)[0])

(33750, 33)
Epoch 0
Loss: 1.116949316319236
Loss: 1.0815245152896282
Loss: 1.0471970289361037
Loss: 1.067686024670261
Loss: 1.021989519254288
Loss: 1.1017323582360166
Epoch 1
Loss: 1.0089989714840717
Loss: 1.0688308278227563
Loss: 1.0502225181227394
Loss: 1.0661093898824543
Loss: 1.0324767350238457
Loss: 1.0898361607061342
Epoch 2
Loss: 1.0233656327299656
Loss: 1.0645191776642977
Loss: 1.0524244848605497
Loss: 1.06291999551147
Loss: 1.0111712000256246
Loss: 1.087391981427789
Epoch 3
Loss: 1.0168849899327186
Loss: 1.072658288285803
Loss: 1.0449283833739034
Loss: 1.0541194812162429
Loss: 1.0120800555459288
Loss: 1.0887429270374627
Epoch 4
Loss: 1.0183565622613742
Loss: 1.0525879598550065
Loss: 1.0564841484096652
Loss: 1.0503410521012064
Loss: 0.9923703316795967
Loss: 1.0755840225507445
Epoch 5
Loss: 1.0201594607570752
Loss: 1.0448586745844024
Loss: 1.0612754018875208
Loss: 1.0510821982505623
Loss: 1.0013734757732768
Loss: 1.0767010674832402
Epoch 6
Loss: 1.0213557644866809
Loss: 1.045637

In [227]:
batch_size = np.shape(train_data)[1]
epochs = 200
learning_rate = 0.001
valid_epoch = 1

model = torchNeuralNetwork(np.shape(train_data)[1], 3)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
best_loss = 10

for e in range(epochs):
  model.train()
  print("Epoch {}".format(e))
  for batch in range(np.shape(train_data)[0]//batch_size):
    pred = model(torch.Tensor(train_data[batch*batch_size : (batch+1)*batch_size]))
    loss = loss_fn(pred, torch.Tensor(train_targets[batch*batch_size : (batch+1)*batch_size]))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if(batch % 100 == 0):
      loss, current = loss.item(), batch * batch_size
      print(f"loss: {loss:>7f}  [{current:>5d}/{np.shape(train_targets)[0]:>5d}]")
  if(e % valid_epoch == 0):
    model.eval()
    with torch.no_grad():
      pred = model(torch.Tensor(val_data))
      loss = loss_fn(pred, torch.Tensor(val_targets)[0])
      print("Validation loss: {:.7f}".format(loss.item()))
      if loss.item() > best_loss:
        break
      best_loss = loss.item()
  print()
  # print(f"loss: {loss:>7f}")

model.eval()
with torch.no_grad():
  pred = model(torch.Tensor(test_data))

  print(pred)
  print(np.sum(test_targets[range(np.shape(test_targets)[0]), np.argmax(np.array(pred), axis=1)])/np.shape(test_targets)[0])



Epoch 0
loss: 0.638477  [    0/16875]
loss: 0.629087  [ 3400/16875]
loss: 0.637131  [ 6800/16875]
loss: 0.610273  [10200/16875]
loss: 0.600855  [13600/16875]
Validation loss: 0.6162647

Epoch 1
loss: 0.625421  [    0/16875]
loss: 0.599668  [ 3400/16875]
loss: 0.635381  [ 6800/16875]
loss: 0.603334  [10200/16875]
loss: 0.599370  [13600/16875]
Validation loss: 0.6154556

Epoch 2
loss: 0.623961  [    0/16875]
loss: 0.598230  [ 3400/16875]
loss: 0.634221  [ 6800/16875]
loss: 0.604294  [10200/16875]
loss: 0.599079  [13600/16875]
Validation loss: 0.6152219

Epoch 3
loss: 0.622742  [    0/16875]
loss: 0.598075  [ 3400/16875]
loss: 0.633393  [ 6800/16875]
loss: 0.605098  [10200/16875]
loss: 0.598907  [13600/16875]
Validation loss: 0.6151251

Epoch 4
loss: 0.621953  [    0/16875]
loss: 0.598074  [ 3400/16875]
loss: 0.632903  [ 6800/16875]
loss: 0.605742  [10200/16875]
loss: 0.598800  [13600/16875]
Validation loss: 0.6150464

Epoch 5
loss: 0.621337  [    0/16875]
loss: 0.598134  [ 3400/16875]
lo